In [28]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import gradio as gr
openai = OpenAI()
claude = anthropic.Anthropic()
import anthropic

In [29]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
OPENAI_MODEL = 'gpt-4o-mini'
ANTHROPIC_MODEL = 'claude-3-5-sonnet-20240620'


In [30]:
system_message = "You are an assistant who reimplements Python code in high performance C++ for an M3 Macbook Pro."
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments."
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [31]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time."
    user_prompt += "Respond only in C++ code; do not explain your work other than occasional comments."
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt
    

In [32]:
def messages_for(python):
    return [
        {"role":"system","content":system_message},
        {"role":"user","content":user_prompt_for(python)},
    ]

In [33]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp","w") as f:
        f.write(code)

In [34]:
def optimize_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)
    

In [42]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=ANTHROPIC_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)
            

In [36]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [37]:
exec(pi)

Result: 3.141592658589
Execution Time: 6.819362 seconds


In [38]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>
#include <cstdint>

double calculate(int64_t iterations, double param1, double param2) {
    double result = 1.0;
    for (int64_t i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100'000'000, 4.0, 1.0) * 4.0;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> elapsed = end_time - start_time;
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << elapsed.count() << " seconds" << std::endl;

    return 0;
}
```

In [39]:
exec(pi)

Result: 3.141592658589
Execution Time: 6.827405 seconds


In [40]:
!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.132717667000 seconds


In [43]:
optimize_claude(pi)

#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        double j = static_cast<double>(i) * param1 - param2;
        result -= 1.0 / j;
        j = static_cast<double>(i) * param1 + param2;
        result += 1.0 / j;
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000LL, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;

    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);
    std::cout << "Execution Time: " << duration.count() / 1e6 << " seconds" << std::endl;

    return 0;
}

In [44]:
!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.137831000000 seconds


GPT 0.132717667000
CLAUDE 0.137831000000